In [1]:
! pip install tess-atlas -q

from tess_atlas.utils import notebook_initalisations

notebook_initalisations()

# TESS Atlas fit for TOI 571

**Version: '0.2.1.dev314+ge1761e8'**

**Note: This notebook was automatically generated as part of the TESS Atlas project. More information can be found on GitHub:** [github.com/dfm/tess-atlas](https://github.com/dfm/tess-atlas)

In this notebook, we do a quicklook fit for the parameters of the TESS Objects of Interest (TOI) in the system number 571.
To do this fit, we use the [exoplanet](https://exoplanet.dfm.io) library and you can find more information about that project at [exoplanet.dfm.io](https://exoplanet.dfm.io).

From here, you can
- scroll down and take a look at the fit results
- open the notebook in Google Colab to run the fit yourself
- download the notebook



## Caveats

There are many caveats associated with this relatively simple "quicklook" type of analysis that should be kept in mind.
Here are some of the main things that come to mind:

1. The orbits that we fit are constrained to be *circular*. One major effect of this approximation is that the fit will significantly overestimate the confidence of the impact parameter constraint, so the results for impact parameter shouldn't be taken too seriously.

2. Transit timing variations, correlated noise, and (probably) your favorite systematics are ignored. Sorry!

3. This notebook was generated automatically without human intervention. Use at your own risk!


## Getting started

To get going, we'll add some _magic_, import some packages, and run some setup steps.

In [2]:
%load_ext autoreload
%load_ext memory_profiler
%load_ext autotime
%autoreload 2
# %matplotlib inline

import os

import numpy as np
import pymc3_ext as pmx
from arviz import InferenceData

from tess_atlas.analysis.eccenticity_reweighting import (
    calculate_eccentricity_weights,
)
from tess_atlas.analysis.model_tools import (
    compute_variable,
    get_untransformed_varnames,
    sample_prior,
)
from tess_atlas.data.inference_data_tools import (
    get_optimized_init_params,
    summary,
    test_model,
)
from tess_atlas.data.tic_entry import TICEntry
from tess_atlas.logger import get_notebook_logger
from tess_atlas.plotting import (
    plot_diagnostics,
    plot_eccentricity_posteriors,
    plot_inference_trace,
    plot_lightcurve,
    plot_phase,
    plot_posteriors,
    plot_priors,
    plot_raw_lightcurve,
)

TOI_NUMBER = 571
logger = get_notebook_logger(outdir=f"toi_{TOI_NUMBER}_files")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 8.57 ms (started: 2023-08-11 13:47:24 +10:00)


In [3]:
import theano

from tess_atlas.utils import tabulate_global_environ_vars

logger.info("Logging some settings for future reference")
logger.info("GLOBAL ENVS:\n" + tabulate_global_environ_vars())
logger.info(f"THEANO Config:\n{theano.config}")

time: 10.5 ms (started: 2023-08-11 13:47:24 +10:00)


## Downloading Data

Next, we grab some inital guesses for the TOI's parameters from [ExoFOP](https://exofop.ipac.caltech.edu/tess/) and download the TOI's lightcurve with [Lightkurve].

We wrap the information in three objects, a `TIC Entry`, a `Planet Candidate` and finally a `Lightcurve Data` object.

- The `TIC Entry` object holds one or more `Planet Candidate`s (each candidate associated with one TOI id number) and a `Lightcurve Data` for associated with the candidates. Note that the `Lightcurve Data` object is initially the same fopr each candidate but may be masked according to the candidate transit's period.

- The `Planet Candidate` holds information on the TOI data collected by [SPOC] (eg transit period, etc)

- The `Lightcurve Data` holds the lightcurve time and flux data for the planet candidates.

[ExoFOP]: https://exofop.ipac.caltech.edu/tess/
[Lightkurve]: https://docs.lightkurve.org/index.html
[SPOC]: https://heasarc.gsfc.nasa.gov/docs/tess/pipeline.html

Downloading the data (this may take a few minutes):

In [4]:
tic_entry = TICEntry.load(toi=TOI_NUMBER)
tic_entry

time: 1.24 s (started: 2023-08-11 13:47:24 +10:00)


TOI,571.01
TOI,571.01
Classification,Known Planet
Period (days),4.641837
Epoch (TBJD),2577.802736
Depth (ppt),7.96
Duration (days),0.155042
Planet SNR,91.0
Single Transit,False
PE Pipeline,QLP
Comments,HAT-P-42 b


If the amount of lightcurve data availible is large we filter the data to keep only data around transits.

In [5]:
if tic_entry.lightcurve.len > 1e5:
    tic_entry.lightcurve.filter_non_transit_data(tic_entry.candidates)
else:
    logger.info("Using the full lightcurve for analysis.")

time: 1.17 ms (started: 2023-08-11 13:47:26 +10:00)


Plot of the lightcurve:

In [6]:
plot_lightcurve(tic_entry, save=True)

# Some diagnostics
plot_raw_lightcurve(tic_entry, save=True)
plot_raw_lightcurve(tic_entry, zoom_in=True, save=True)

time: 2.56 s (started: 2023-08-11 13:47:26 +10:00)


![](toi_571_files/flux_vs_time.png)

Diagnostic plots of the raw lightcurve (not applying sigma clipping/other cleaning methods to remove outliers). Some things to consider:
- Do the initial fits from ExoFOP match the transits if visible?
- If this is marked as a single-transit event, is there only 1 transit visible?

![](toi_571_files/diagnostic_raw_lc_plot.png)
![](toi_571_files/diagnostic_raw_lc_plot_zoom.png)

## Fitting transit parameters
Now that we have the data, we can define a Bayesian model to fit it.

### The probabilistic model

We use the probabilistic model as described in [Foreman-Mackey et al 2017] to determine the best parameters to fit the transits present in the lightcurve data.

More explicitly, the stellar light curve $l(t; \vec{\theta})$ is modelled with a Gaussian Process (GP).
A GP consists of a mean function $\mu(t;\vec{\theta})$ and a kernel function $k_\alpha(t,t';\vec{\theta})$, where $\vec{\theta}$ is the vector of parameters descibing the lightcurve and $t$ is the time during which the lightcurve is under observation

The 8 parameters describing the lightcurve are
$$\vec{\theta} = \{d_i, t0_i, tmax_i, b_i, r_i, f0, u1, u2\},$$
where
* $d_i$ transit durations for each planet,
* $tmin_i$ time of first transit for each planet (reference time),
* $tmax_i$ time of the last transit for each planet (a second reference time),
* $b_i$ impact parameter for each planet,
* $r_i$ planet radius in stellar radius for each planet,
* $f0$ baseline relative flux of the light curve from star,
* $u1$ $u2$ two parameters describing the limb-darkening profile of star.

Note: if the observed data only records a single transit,
we swap $tmax_i$ with $p_i$ (orbital periods for each planet).

With this we can write
$$l(t;\vec{\theta}) \sim \mathcal{GP} (\mu(t;\vec{\theta}), k_\alpha(t,t';\vec{\theta}))\ .$$

Here the mean and kernel functions are:
* $\mu(t;\vec{\theta})$: a limb-darkened transit light curve ([Kipping 2013])
* $k_\alpha(t,t';\vec{\theta}))$: a stochastically-driven, damped harmonic oscillator ([SHOTterm])


Now that we have defined our transit model, we can implement it in python (toggle to show).

[Foreman-Mackey et al 2017]: https://arxiv.org/pdf/1703.09710.pdf
[Kipping 2013]: https://arxiv.org/abs/1308.0009
[SHOTterm]: https://celerite2.readthedocs.io/en/latest/api/python/?highlight=SHOTerm#celerite2.terms.SHOTerm

In [7]:
import aesara_theano_fallback.tensor as tt
import exoplanet as xo
import numpy as np
import pymc3 as pm
import pymc3_ext as pmx
from celerite2.theano import GaussianProcess, terms

DEPTH = "depth"
DURATION = "dur"
RADIUS_RATIO = "r"
TIME_START = "tmin"
TIME_END = "tmax"
ORBITAL_PERIOD = "p"
MEAN_FLUX = "f0"
LC_JITTER = "jitter"
GP_RHO = "rho"
GP_SIGMA = "sigma"
RHO_CIRC = "rho_circ"  # stellar density at e=0
LIMB_DARKENING_PARAM = "u"
IMPACT_PARAM = "b"


def get_test_duration(min_durations, max_durations, durations):
    largest_min_duration = np.amax(
        np.array([durations, 2 * min_durations]), axis=0
    )
    smallest_max_duration = np.amin(
        np.array([largest_min_duration, 0.99 * max_durations]), axis=0
    )
    return smallest_max_duration


def build_planet_transit_model(tic_entry):
    t = tic_entry.lightcurve.time
    y = tic_entry.lightcurve.flux
    yerr = tic_entry.lightcurve.flux_err

    n = tic_entry.planet_count
    tmins = np.array([planet.tmin for planet in tic_entry.candidates])
    depths = np.array([planet.depth for planet in tic_entry.candidates])
    durations = np.array([planet.duration for planet in tic_entry.candidates])
    max_durations = np.array(
        [planet.duration_max for planet in tic_entry.candidates]
    )
    min_durations = np.array(
        [planet.duration_min for planet in tic_entry.candidates]
    )
    test_duration = get_test_duration(min_durations, max_durations, durations)

    with pm.Model() as my_planet_transit_model:
        ## define planet parameters

        # 1) d: transit duration (duration of eclipse)
        d_priors = pm.Bound(
            pm.Lognormal, lower=min_durations, upper=max_durations
        )(
            name=DURATION,
            mu=np.log(durations),
            sigma=np.log(1.2),
            shape=n,
            testval=test_duration,
        )

        # 2) r: radius ratio (planet radius / star radius)
        r_priors = pm.Lognormal(
            name=RADIUS_RATIO, mu=0.5 * np.log(depths * 1e-3), sd=1.0, shape=n
        )
        # 3) b: impact parameter
        b_priors = xo.distributions.ImpactParameter(
            name=IMPACT_PARAM, ror=r_priors, shape=n
        )
        planet_priors = [r_priors, d_priors, b_priors]

        ## define orbit-timing parameters

        # 1) tmin: the time of the first transit in data (a reference time)
        tmin_norm = pm.Bound(
            pm.Normal, lower=tmins - max_durations, upper=tmins + max_durations
        )
        tmin_priors = tmin_norm(
            TIME_START, mu=tmins, sigma=0.5 * durations, shape=n, testval=tmins
        )

        # 2) period: the planets' orbital period
        p_params, p_priors_list, tmax_priors_list = [], [], []
        for n, planet in enumerate(tic_entry.candidates):
            # if only one transit in data we use the period
            if planet.has_data_only_for_single_transit:
                p_prior = pm.Pareto(
                    name=f"{ORBITAL_PERIOD}_{planet.index}",
                    m=planet.period_min,
                    alpha=2.0 / 3.0,
                    testval=planet.period,
                )
                p_param = p_prior
                tmax_prior = planet.tmin
            # if more than one transit in data we use a second time reference (tmax)
            else:
                tmax_norm = pm.Bound(
                    pm.Normal,
                    lower=planet.tmax - planet.duration_max,
                    upper=planet.tmax + planet.duration_max,
                )
                tmax_prior = tmax_norm(
                    name=f"{TIME_END}_{planet.index}",
                    mu=planet.tmax,
                    sigma=0.5 * planet.duration,
                    testval=planet.tmax,
                )
                p_prior = (tmax_prior - tmin_priors[n]) / planet.num_periods
                p_param = tmax_prior

            p_params.append(p_param)  # the param needed to calculate p
            p_priors_list.append(p_prior)
            tmax_priors_list.append(tmax_prior)

        p_priors = pm.Deterministic(ORBITAL_PERIOD, tt.stack(p_priors_list))
        tmax_priors = pm.Deterministic(TIME_END, tt.stack(tmax_priors_list))

        ## define stellar parameters

        # 1) f0: the mean flux from the star
        f0_prior = pm.Normal(name=MEAN_FLUX, mu=0.0, sd=10.0)

        # 2) u1, u2: limb darkening parameters
        u_prior = xo.distributions.QuadLimbDark("u")
        stellar_priors = [f0_prior, u_prior]

        ## define k(t, t1; parameters)
        jitter_prior = pm.InverseGamma(
            name=LC_JITTER, **pmx.estimate_inverse_gamma_parameters(1.0, 5.0)
        )
        sigma_prior = pm.InverseGamma(
            name=GP_SIGMA, **pmx.estimate_inverse_gamma_parameters(1.0, 5.0)
        )
        rho_prior = pm.InverseGamma(
            name=GP_RHO, **pmx.estimate_inverse_gamma_parameters(0.5, 10.0)
        )
        kernel = terms.SHOTerm(sigma=sigma_prior, rho=rho_prior, Q=0.3)
        noise_priors = [jitter_prior, sigma_prior, rho_prior]

        ## define the lightcurve model mu(t;paramters)
        orbit = xo.orbits.KeplerianOrbit(
            period=p_priors,
            t0=tmin_priors,
            b=b_priors,
            duration=d_priors,
            ror=r_priors,
        )
        star = xo.LimbDarkLightCurve(u_prior)
        lightcurve_models = star.get_light_curve(orbit=orbit, r=r_priors, t=t)
        lightcurve = 1e3 * pm.math.sum(lightcurve_models, axis=-1) + f0_prior
        my_planet_transit_model.lightcurve_models = lightcurve_models
        rho_circ = pm.Deterministic(name=RHO_CIRC, var=orbit.rho_star)

        # Finally the GP likelihood
        residual = y - lightcurve
        gp_kwargs = dict(diag=yerr**2 + jitter_prior**2, quiet=True)
        gp = GaussianProcess(kernel, t, **gp_kwargs)
        gp.marginal(name="obs", observed=residual)
        my_planet_transit_model.gp_mu = gp.predict(residual, return_var=False)

        # cache params
        my_params = dict(
            planet_params=planet_priors,
            noise_params=noise_priors,
            stellar_params=stellar_priors,
            period_params=p_params,
        )
    return my_planet_transit_model, my_params

time: 162 ms (started: 2023-08-11 13:47:28 +10:00)


In [8]:
planet_transit_model, params = build_planet_transit_model(tic_entry)
model_varnames = get_untransformed_varnames(planet_transit_model)
test_model(planet_transit_model)

time: 4min 13s (started: 2023-08-11 13:47:29 +10:00)


### Optimizing the initial point for sampling
We help out the sampler we try to find an optimized set of initial parameters to begin sampling from.

In [9]:
if tic_entry.optimized_params is None:
    init_params = get_optimized_init_params(planet_transit_model, **params)
    tic_entry.save_data(optimized_params=init_params)
else:
    init_params = tic_entry.optimized_params.to_dict()

time: 1.19 ms (started: 2023-08-11 13:51:42 +10:00)


In [10]:
# sanity check that none of the right hand column have nans!
test_model(planet_transit_model, init_params, show_summary=True)

time: 2.32 s (started: 2023-08-11 13:51:42 +10:00)


,Test Point,log P(test-point)
dur_interval__,-2.215,0.63
r_log__,-2.417,-0.93
b_impact__,-0.164,-1.73
tmin_interval__,0.0,1.38
tmax_1_interval__,0.0,1.38
f0,0.0,-3.22
u_quadlimbdark__,0.0,-3.37
jitter_log__,0.556,-46.54
sigma_log__,0.556,-23.52
rho_log__,0.053,-3.20


Below are plots of our initial model and priors.

### Initial model fit

In [11]:
initial_lc_models = (
    compute_variable(
        model=planet_transit_model,
        samples=[[init_params[n] for n in model_varnames]],
        target=planet_transit_model.lightcurve_models,
    )
    * 1e3
)
plot_lightcurve(
    tic_entry, initial_lc_models, save="lightcurve_with_initial_guess.png"
)
plot_lightcurve(
    tic_entry,
    initial_lc_models,
    zoom_in=True,
    save="lightcurve_with_initial_guess_zoom.png",
)

time: 4.97 s (started: 2023-08-11 13:51:45 +10:00)


<!-- Show LC plot with initial guess -->
![](toi_571_files/lightcurve_with_initial_guess.png)

<!-- Show zoomed in LC plot with initial guess -->
![](toi_571_files/lightcurve_with_initial_guess_zoom.png)

In [12]:
params = dict(
    tic_entry=tic_entry, model=planet_transit_model, initial_params=init_params
)
plot_phase(**params, save="phase_initial.png")
plot_phase(
    **params, plot_all_datapoints=True, save="phase_initial_all_datapoints.png"
)

time: 3 s (started: 2023-08-11 13:51:50 +10:00)


<!-- SHOW PHASE PLOT -->
<img src="toi_571_files/phase_initial.png" style="width:450px;"/>


Diagnostic phase plot
<img src="toi_571_files/phase_initial_all_datapoints.png" style="width:450px;"/>

### Histograms of Priors

In [13]:
prior_samples = sample_prior(planet_transit_model)
if prior_samples:
    plot_priors(tic_entry, prior_samples, init_params, save=True)

time: 9.44 s (started: 2023-08-11 13:51:53 +10:00)


![](toi_571_files/priors.png)


### Sampling
With the model and priors defined, we can begin sampling.

In [14]:
def run_inference(model) -> InferenceData:
    np.random.seed(TOI_NUMBER)
    with model:
        sampling_kwargs = dict(tune=2000, draws=2000, chains=2, cores=2)
        logger.info(f"Run sampler with kwargs: {sampling_kwargs}")
        inference_data = pmx.sample(
            **sampling_kwargs, start=init_params, return_inferencedata=True
        )
        logger.info("Sampling completed!")
        return inference_data

time: 1.14 ms (started: 2023-08-11 13:52:03 +10:00)


In [15]:
if tic_entry.inference_data is None:
    inference_data = run_inference(planet_transit_model)
    tic_entry.inference_data = inference_data
    tic_entry.save_data(inference_data=inference_data)
else:
    logger.info("Using cached run")
    inference_data = tic_entry.inference_data
inference_data


time: 80.5 ms (started: 2023-08-11 13:52:03 +10:00)


Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats

The `inference_data` object contains the posteriors and sampling metadata. Let's save it for future use, and take a look at summary statistics. Note: the _trace plot_ from sampling is hidden below.

In [16]:
summary(inference_data)

time: 223 ms (started: 2023-08-11 13:52:03 +10:00)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
f0,0.104,0.024,0.061,0.148,0.001,0.000,1984.0,1585.0,1.00
dur[0],0.161,0.001,0.159,0.164,0.000,0.000,760.0,700.0,1.00
r[0],0.081,0.001,0.078,0.083,0.000,0.000,473.0,222.0,1.00
b[0],0.254,0.138,0.007,0.473,0.007,0.005,372.0,328.0,1.00
tmin[0],1519.463,0.001,1519.462,1519.464,0.000,0.000,1594.0,886.0,1.00
tmax_1,2986.285,0.000,2986.284,2986.286,0.000,0.000,2203.0,2104.0,1.00
p[0],4.642,0.000,4.642,4.642,0.000,0.000,2060.0,2174.0,1.00
tmax[0],2986.285,0.000,2986.284,2986.286,0.000,0.000,2203.0,2104.0,1.00
u[0],0.294,0.121,0.073,0.522,0.004,0.003,759.0,723.0,1.01
u[1],0.378,0.234,-0.066,0.785,0.008,0.006,789.0,532.0,1.00


In [17]:
plot_inference_trace(tic_entry, save=True)

time: 4.49 s (started: 2023-08-11 13:52:03 +10:00)


![](toi_571_files/diagnostic_trace_plot.png)

## Results


### Posterior plots
Below are plots of the posterior probability distributions and the best-fitting light-curve model.

In [18]:
plot_posteriors(
    tic_entry, inference_data, initial_params=init_params, save=True
)

time: 5.29 s (started: 2023-08-11 13:52:08 +10:00)


<!-- SHOW POSTERIOR PLOT -->
![](toi_571_files/posteriors.png)

In [19]:
%%memit
plot_phase(
    tic_entry,
    planet_transit_model,
    inference_data,
    initial_params=init_params,
    save=True,
)

peak memory: 432.83 MiB, increment: 8.54 MiB
time: 10.6 s (started: 2023-08-11 13:52:13 +10:00)


<!-- SHOW PHASE PLOT -->
<img src="toi_571_files/phase_plot.png" style="width:450px;"/>

### Eccentricity post-processing

As discussed above, we fit this model assuming a circular orbit which speeds things up for a few reasons:
1) `e=0` allows simpler orbital dynamics which are more computationally efficient (no need to solve Kepler's equation numerically)

2) There are degeneracies between eccentricity, arrgument of periasteron, impact parameter, and planet radius. Hence by setting `e=0` and using the duration in calculating the planet's orbit, the sampler can perform better.

To first order, the eccentricity mainly just changes the transit duration.
This can be thought of as a change in the impled density of the star.
Therefore, if the transit is fit using stellar density (or duration, in this case) as one of the parameters, it is possible to make an independent measurement of the stellar density, and in turn infer the eccentricity of the orbit as a post-processing step.
The details of this eccentricity calculation method are described in [Dawson & Johnson (2012)].

Here, if the TIC has associated stellar data, we use the method described above to obtain fits for the exoplanet's orbital eccentricity.

[Dawson & Johnson (2012)]: https://arxiv.org/abs/1203.5537
Note: a different stellar density parameter is required for each planet (if there is more than one planet)

In [20]:
star = tic_entry.stellar_data
star

time: 1.97 ms (started: 2023-08-11 13:52:24 +10:00)


In [21]:
if star.density_data_present:
    logger.info(
        "Stellar data present for TIC. Continuing with eccentricity calculations."
    )
else:
    logger.info(
        "Stellar data not present for TIC. Skipping eccentricity calculations."
    )

time: 1.3 ms (started: 2023-08-11 13:52:24 +10:00)


In [22]:
if star.density_data_present:
    ecc_samples = calculate_eccentricity_weights(tic_entry, inference_data)
    ecc_samples.to_csv(
        os.path.join(tic_entry.outdir, "eccentricity_samples.csv"), index=False
    )
    plot_eccentricity_posteriors(tic_entry, ecc_samples, save=True)

time: 594 ms (started: 2023-08-11 13:52:24 +10:00)


<!-- SHOW ECC POSTERIORS -->
![](toi_571_files/eccentricity_posteriors.png)

### Diagnostics
Finally, we also generate some diagnostic plots.

In [23]:
plot_diagnostics(tic_entry, planet_transit_model, init_params, save=True)

time: 9.43 s (started: 2023-08-11 13:52:25 +10:00)


<!-- SHOW DIAGNOSTICS -->

![](toi_571_files/diagnostic_flux_vs_time_zoom.png)

## Citations

We hope this has been helpful! The TESS-Atlas was built using exoplanet, PyMC3, lightkurve, starry, celerite2, ExoFOP, and Sphinx.

We would greatly appreciate you citing this work and its dependencies.

### LaTeX acknowledgement and bibliography

In [24]:
from tess_atlas import citations

citations.print_acknowledgements()

This research made use of the \textsf{TESS-Atlas} and its dependencies
\citep{exoplanet:joss, exoplanet:zenodo, celerite2:foremanmackey17,
celerite2:foremanmackey18, exoplanet:agol20, exoplanet:arviz,
exoplanet:astropy13, exoplanet:astropy18, exoplanet:kipping13,
exoplanet:luger18, exoplanet:pymc3, exoplanet:theano}.

time: 4.61 ms (started: 2023-08-11 13:52:34 +10:00)


In [25]:
citations.print_bibliography()

@article{exoplanet:joss,
       author = {{Foreman-Mackey}, Daniel and {Luger}, Rodrigo and {Agol}, Eric
                and {Barclay}, Thomas and {Bouma}, Luke G. and {Brandt},
                Timothy D. and {Czekala}, Ian and {David}, Trevor J. and
                {Dong}, Jiayin and {Gilbert}, Emily A. and {Gordon}, Tyler A.
                and {Hedges}, Christina and {Hey}, Daniel R. and {Morris},
                Brett M. and {Price-Whelan}, Adrian M. and {Savel}, Arjun B.},
        title = "{exoplanet: Gradient-based probabilistic inference for
                  exoplanet data \& other astronomical time series}",
      journal = {arXiv e-prints},
         year = 2021,
        month = may,
          eid = {arXiv:2105.01994},
        pages = {arXiv:2105.01994},
archivePrefix = {arXiv},
       eprint = {2105.01994},
 primaryClass = {astro-ph.IM},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021arXiv210501994F},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}

### Packages used


In [26]:
citations.print_packages()

zipp==3.6.0
xarray==0.19.0
widgetsnbextension==3.5.1
wheel==0.37.0
websocket-client==1.2.1
webencodings==0.5.1
wcwidth==0.2.5
virtualenv==20.8.1
urllib3==1.26.7
uncertainties==3.1.6
uc-micro-py==1.0.1
typing-extensions==3.10.0.2
traitlets==5.1.0
tqdm==4.62.3
tornado==6.1
tomli==1.2.1
toml==0.10.2
tinycss2==1.2.1
threadpoolctl==3.0.0
Theano-PyMC==1.1.2
testpath==0.5.0
testbook==0.4.2
tess-atlas==0.2.1.dev314+ge1761e8
terminado==0.12.1
tenacity==8.0.1
tar-progress==1.3.3
tabulate==0.8.10
tables==3.7.0
StrEnum==0.4.15
SQLAlchemy==1.4.26
sphinxcontrib-serializinghtml==1.1.5
sphinxcontrib-qthelp==1.0.3
sphinxcontrib-jsmath==1.0.1
sphinxcontrib-htmlhelp==2.0.0
sphinxcontrib-devhelp==1.0.2
sphinxcontrib-bibtex==2.2.1
sphinxcontrib-applehelp==1.0.2
Sphinx==5.0.2
sphinx-toolbox==3.1.2
sphinx-togglebutton==0.2.2
sphinx-thebe==0.2.1
sphinx-tabs==1.2.1
sphinx-remove-toctrees==0.0.3
sphinx-prompt==1.5.0
sphinx-panels==0.6.0
sphinx-multitoc-numbering==0.1.3
sphinx-jupyterbook-latex==0.5.2
sphinx-jin

## Comments
Leave a comment below or in this [issue](https://github.com/avivajpeyi/tess-atlas/issues/new?title=TOI571).
```{raw} html
<script src="https://utteranc.es/client.js"
        repo="avivajpeyi/tess-atlas"
        issue-term="TOI571"
        theme="github-light"
        crossorigin="anonymous"
        async>
</script>
```